In [ ]:
import requests
import pandas as pd

url = "https://moviesdatabase.p.rapidapi.com/titles?"

parametros =  {"genre":"Drama","startYear":"1998","endYear":"2006","page":"1"}

TOKEN=input("Introduce tu token secreto")

headers = {
	"X-RapidAPI-Key": TOKEN,
	"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=parametros)

print(response.json())

respuesta_json = response.json()




In [ ]:
print(respuesta_json["results"][0])

In [ ]:
# solo me da una pagina, 
# Necesitamos hacer una función y que esta función saque una lista de tuplas.


def convertir_entrada_tupla(entry):
    tipo = entry["titleType"]["id"]
    nombre = entry["originalTitleText"]["text"]
    año = 0
    if entry["releaseDate"] == None:
        año = entry["releaseYear"]["year"]
    else:
        año = entry["releaseDate"]["year"]
    mes = 0
    if entry["releaseDate"] == None:
        mes = 0
    else:
        mes = entry["releaseDate"]["month"]    
    id = entry["id"]
    tupla = (tipo, nombre, año, mes, id)

    # tupla tiene el siguiente formato: ("Pelicula", "Titanic", 1997, 12, "abc123")
    return tupla

#convertir_entrada_tupla(respuesta_json["results"][3])

In [ ]:
# quiero que SOLO me muestre peliculas (movie, short, tvMovie)

def entrada_es_pelicula(entry):
    if entry["titleType"]["id"] in ["movie","short","tvMovie"]:
        return True
    else:
        return False

In [ ]:
def resultados_a_tuplas(resultados):
    # para cada una de las entradas en pagina["results"]
    lista_tuplas = [ convertir_entrada_tupla(entry) for entry in resultados if entrada_es_pelicula(entry)]
    return lista_tuplas

#resultados_a_tuplas(respuesta_json["results"])

In [40]:
# queremos que nos vaya por las pagínas hasta que next = none y menos de 50 porque en la página 50 se vuelve loco

url = "https://moviesdatabase.p.rapidapi.com"
next = "/titles?endYear=2006&startYear=1998&genre=Drama&page=2"

def recorrer_paginas(pagina):

    print(pagina["next"])
    print()
    lista_tuplas = resultados_a_tuplas(pagina["results"])
   

    while pagina["next"] != None and int(pagina["page"]) < 10:
        url_siguientepagina = url + pagina["next"]
        print (url_siguientepagina)
        response = requests.get(url_siguientepagina, headers=headers)
        respuesta_json = response.json()
        print(respuesta_json)

        lista_tuplas= lista_tuplas + (resultados_a_tuplas(respuesta_json["results"]))
        
        pagina_nueva = respuesta_json
        pagina = pagina_nueva


    return lista_tuplas

#recorrer_paginas(respuesta_json)

In [41]:
# podríamos poner esto en una función que reciba el parámetro "genero"

def sacar_peliculas_genero(genero):

    url = "https://moviesdatabase.p.rapidapi.com/titles?"
    
    parametros =  {"genre":genero,"startYear":"1998","endYear":"2006","page":"1"}

    headers = {
        "X-RapidAPI-Key": TOKEN,
        "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=parametros)

    print(response.json())

    respuesta_json = response.json()
    pagina = respuesta_json

    lista_tuplas = recorrer_paginas(pagina)

    df = pd.DataFrame(lista_tuplas,columns=["Tipo","Nombre","Año estreno","Mes estreno","Id"])
    df.to_csv(f"peliculas_1998-2006/peliculas_1998-2006_{genero}.csv")

    return lista_tuplas

#lista_tuplas = sacar_peliculas_genero("Comedy")

In [42]:
def sacar_peliculas():
    lista_genero= ['Drama','Comedy', 'Action', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Thriller']
    lista_tuplas = []

    for genero in lista_genero:
        lista_tuplas = lista_tuplas + sacar_peliculas_genero(genero)
    return lista_tuplas    

lista_tuplas = sacar_peliculas()    

{'page': '1', 'next': '/titles?endYear=2006&page=2&startYear=1998&genre=Drama', 'entries': 10, 'results': [{'_id': '61e58179cffb8f2faa41662f', 'id': 'tt0110476', 'primaryImage': {'id': 'rm941448192', 'width': 626, 'height': 862, 'url': 'https://m.media-amazon.com/images/M/MV5BMjFhMDFiZTctMDRmZC00ZmZlLWIwN2EtM2YwMWZmZjE1YjI3XkEyXkFqcGdeQXVyNjMxMzM3NDI@._V1_.jpg', 'caption': {'plainText': 'Master i Margarita (2006)', '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, '__typename': 'TitleType'}, 'titleText': {'text': 'Master i Margarita', '__typename': 'TitleText'}, 'originalTitleText': {'text': 'Master i Margarita', '__typename': 'TitleText'}, 'releaseYear': {'year': 2006, 'endYear': None, '__typename': 'YearRange'}, 'releaseDate': {'day': 7, 'month': 4, 'year': 2011, '__typename': 'ReleaseDate'}}, {'_id': '61e581b6d735dff3f94171c9', 'id': 'tt0129871', 'primaryImage': {'id': 'rm713986560', 'width': 334, 

In [ ]:
#Esto es para sacar todfas las peliculas de esos años de todos los género, pero se peta, así que arriba lo haré de 10 paginas por genero

#print(len(lista_tuplas))
#df = pd.DataFrame(lista_tuplas,columns=["Tipo","Nombre","Año estreno","Mes estreno","Id"])
#df.to_csv("peliculas_1998-2006.csv")